# Nexus results exploration
In this notebook, some Nexus interactions from the modelling results can be explored. This notebook is inteded to be used as a tool for quickly visualize results from  recent modelling effort. However, for a deeper and interactive exploration of the Nexus, go to the online visualization platform of the Souss-Massa Nexus project.

In [ ]:
#  %matplotlib inline
# import geopandas as gpd
import pandas as pd
# import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
import os
import pickle
import plotly.io as pio

pio.templates.default = "plotly_white"
# import time

## Input data
Provide the scenario names, climatic condition and folder paths to read the data.

In [ ]:
scenario1 = 'Reference'
scenario2 = 'Desalination'
scenario3 = 'Desalination Wastewater Reuse'
climate = 'Climate Change'
# climate_title = f'Historical climate {climate}'
climate_title = climate
folder1 = f"test dash results/data/{scenario1}/{climate}/"
folder2 = f"test dash results/data/{scenario2}/{climate}/"
folder3 = f"test dash results/data/{scenario3}/{climate}/"
df1 = pd.read_csv(f"{folder1}/results.gz")
df2 = pd.read_csv(f"{folder2}/results.gz")
df3 = pd.read_csv(f"{folder3}/results.gz")
df1['Scenario'] = scenario1
df2['Scenario'] = scenario2
df3['Scenario'] = scenario3
df = df1.append(df2)
df = df.append(df3)
df = df.loc[df.Year >= 2020]

## Energy requirements per cubic meter pumped for agricultural irrigation divided by province

In [ ]:
dff = df.loc[df['type'].str.contains('Agriculture')].groupby(['Scenario', 'Province', 'Year'])[['sswd', 'swpa_e']].sum()/1000000
dff['E_int'] = dff.swpa_e/dff.sswd
dff.reset_index(inplace=True)
fig = px.line(dff, x='Year', y='E_int', color='Scenario', facet_col='Province', facet_col_wrap=2, 
              labels={"E_int": "GWh/m3"}, title='Energy requirements per m3 pumped in agriculture',
              facet_col_spacing=0.05)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

## Energy requirements for agricultural irrigation divided by province

In [ ]:
dff = df.loc[df['type'].str.contains('Agriculture')].groupby(['Scenario', 'Province', 'Year'])[['sswd', 'swpa_e']].sum()/1000000
dff.reset_index(inplace=True)

fig = px.line(dff, x='Year', y='swpa_e', color='Scenario', facet_col='Province', facet_col_wrap=2,
              labels={"swpa_e": "Energy demand (GWh)"}, title='Energy demand for water pumping in agriculture',
              facet_col_spacing=0.05)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

## Water requirements for agricultural irrigation divided by province

In [ ]:
dff = df.loc[df['type'].str.contains('Agriculture')].groupby(['Scenario', 'Province', 'Year'])[['sswd', 'swpa_e']].sum()/1000000
dff.reset_index(inplace=True)

fig = px.line(dff, x='Year', y='sswd', color='Scenario', facet_col='Province', facet_col_wrap=2,
              labels={"sswd": "Water demand (m3)"}, title='Water demand for agriculture irrigation',
              facet_col_spacing=0.05)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

In [ ]:
def energy_demand_plot(water_delivered, wwtp_data, desal_data, time_frame):
    dff_energy = water_delivered.copy()
    dff_energy.loc[dff_energy['type'].str.contains('GW'), 'type'] = 'Groundwater pumping'
    dff_energy.loc[dff_energy['type'].str.contains('SW|Pipeline'), 'type'] = 'Surface water conveyance'
    dff_energy.loc[dff_energy['type'].str.contains('DS'), 'type'] = 'Desalinated water conveyance'
    dff_energy.loc[dff_energy['type'].str.contains('WWR'), 'type'] = 'Wastewater reuse conveyance'
    dff_energy = dff_energy.groupby(['Scenario', time_frame, 'type', 'Reuse'])['swpa_e'].sum() / 1000000
    dff_energy = dff_energy.reset_index()
    wwtp_data['type'] = 'Wastewater treatment'
    desal_data['type'] = 'Desalination energy'
    for df in [wwtp_data, desal_data]:
        dff = df.groupby(['Scenario', time_frame, 'type', 'Reuse'])['swpa_e'].sum() / 1000000
        dff = dff.reset_index()
        dff_energy = dff_energy.append(dff, sort=False)
    for t in dff_energy['type'].unique():
        if dff_energy.loc[dff_energy['type']==t, 'swpa_e'].sum() == 0:
            dff_energy = dff_energy.loc[dff_energy['type']!=t]
    return dff_energy

In [ ]:
def water_supply_plot(water_delivered, time_frame):
    dff_delivered = water_delivered.copy()
    dff_delivered.loc[dff_delivered['type'].str.contains('SW|Aquifer'), 'type'] = 'Surface water'
    dff_delivered.loc[dff_delivered['type'].str.contains('GW'), 'type'] = 'Groundwater'
    dff_delivered.loc[dff_delivered['type'].str.contains('DS'), 'type'] = 'Desalinated water'
    dff_delivered.loc[dff_delivered['type'].str.contains('WWR'), 'type'] = 'Reused Wastewater'
    dff_delivered = dff_delivered.loc[dff_delivered['type'] != 'Transmission Pipeline']
    dff_delivered = dff_delivered.groupby(['Scenario', time_frame, 'type', 'Reuse'])['sswd'].sum() / 1000000
    dff_delivered = dff_delivered.reset_index()

    df = dff_delivered.reset_index()
    
    for t in df['type'].unique():
        if df.loc[df['type']==t, 'sswd'].sum() == 0:
            df = df.loc[df['type']!=t]
    return df

In [ ]:
def unmet_demand_plot(water_delivered, time_frame):
    dff_unmet = water_delivered.copy()
    dff_unmet.loc[dff_unmet['type'].str.contains('Agriculture'), 'category'] = 'Agriculture'
    dff_unmet.loc[dff_unmet['type'].str.contains('Domestic'), 'category'] = 'Domestic'
    dff_unmet = dff_unmet.loc[~dff_unmet['type'].str.contains('Aquifer')]
    dff_unmet = dff_unmet.loc[dff_unmet['type'] != 'Transmission Pipeline']
    water_req_year = \
        dff_unmet.groupby(['Scenario','Year', 'Date', 'Demand point', 'category'])['water_required'].mean().reset_index().groupby(
            ['Scenario', time_frame, 'category'])['water_required'].sum()
    unment_demand =pd.DataFrame()
    unment_demand['value'] = 1 - (dff_unmet.groupby(['Scenario', time_frame, 'category'])['sswd'].sum() /
                         water_req_year)

    df = unment_demand.reset_index()
    return df

In [ ]:
import sys
sys.path.append("../Morocco dashboard")
import scripts.read_data as rd

In [ ]:
path = '../Morocco dashboard'
scenarios = [
    'Reference', 
    'Desalination', 
#     'Desalination Wastewater Reuse', 
#     'Increased Water Productivity'
]
climates = [
#     'Trend', 
    'Climate Change'
]
phaseout_year = 2040
pv_level = 20
files = ['results.gz', 'wwtp_data.gz', 'desal_data.gz',
         'summary_results.gz', 'production.gz']
climates_dict = {'Trend': 'Historical climate', 'Climate Change': 'Climate change'}

df_main = pd.DataFrame()
df_wwt = pd.DataFrame()
df_desal = pd.DataFrame()
df_crop = pd.DataFrame()
for scenario in scenarios:
    for climate in climates:
        _df_main, _df_wwt, _df_desal, _df_butane, _df_crop = rd.load_data(path, scenario, [climate], phaseout_year, 
                                                                pv_level, files, from_server=False)
        for dff in [_df_main, _df_wwt, _df_desal, _df_butane, _df_crop]: 
            dff['Scenario'] = scenario
            dff['Climate'] = climates_dict[climate]
        df_main = df_main.append(_df_main, ignore_index=True)
        df_wwt = df_wwt.append(_df_wwt, ignore_index=True)
        df_desal = df_desal.append(_df_desal, ignore_index=True)
        df_crop = df_crop.append(_df_crop, ignore_index=True)

In [ ]:
names = {'sswd': 'Water delivered (Mm<sup>3</sup>)', 'production_kg': 'Crop production (ton)', 
         'swpa_e': 'Energy demand (GWh)', 'unmet_demand_year': 'Annual unmet demand (%)',
         'wtd': 'Depth to groundwater (meters)'}

layout = dict(
    autosize=True,
    height=500,
    margin=dict(l=0, r=0, b=0, t=0),
    hovermode="closest",
    plot_bgcolor="#fff",
    paper_bgcolor="#fff",
    legend=dict(font=dict(size=16), orientation="h", title=''),
    xaxis={'title': ''},
    showlegend=True,
    hoverlabel={'align': 'left'},
    font=dict(size=18, color="#7f7f7f"),
#     yaxis={'title': ''},
)

category_orders = {'Climate': ['Historical climate', 'Climate change'],
                   'Scenario': ['Reference', 'Desalination'],
                   'type': ['Groundwater pumping', 'Surface water conveyance', 'Wastewater treatment',
                            'Desalination', 'Desalinated water conveyance']}

def update_labels(fig):
    for axis in fig.layout:
        if type(fig.layout[axis]) == go.layout.XAxis:
            fig.layout[axis].title.text = ''
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

In [ ]:
dff_unmet = df_main.copy()
dff_unmet.loc[dff_unmet['type'].str.contains('Agriculture'), 'category'] = 'Agriculture'
dff_unmet.loc[dff_unmet['type'].str.contains('Domestic'), 'category'] = 'Domestic'
dff_unmet = dff_unmet.loc[~dff_unmet['type'].str.contains('Aquifer')]
dff_unmet = dff_unmet.loc[dff_unmet['type'] != 'Transmission Pipeline']
water_req_year = \
    dff_unmet.groupby(['Scenario', 'Year', 'Date', 'category', 'Demand point',])[
        'water_required'].mean().reset_index().groupby(
        ['Scenario', 'Year', 'category', 'Demand point'])['water_required'].sum()
unment_demand = pd.DataFrame()
unment_demand['unmet_demand_year'] = 1 - (dff_unmet.groupby(['Scenario', 'Year', 'category', 'Demand point'])['sswd'].sum() /
                              water_req_year)

dff = unment_demand.reset_index()
dff.loc[dff['unmet_demand_year'] < 0, 'unmet_demand_year'] = 0
dff = dff.loc[dff['category']=='Agriculture']
dff = dff.loc[dff['Demand point'].isin([
    'I_Modern Public Massa','I_Moderne Prive Massa','I_Modern Prive Chtouka',
#     'I_Moderne Prive Souss', 'I_Traditional Rehabilite du Souss Perimetre Diffus'
])]

fig = px.line(dff.loc[dff['Scenario'] == 'Reference'], 
              x='Year', y='unmet_demand_year', color='Demand point', 
              color_discrete_sequence=px.colors.qualitative.G10,
              facet_col_spacing=0.06, labels=names, category_orders=category_orders)
fig.update_traces(line=dict(dash='dash'))

fig.add_traces(px.line(dff.loc[dff['Scenario'] == 'Desalination'],
                       x='Year', y='unmet_demand_year', color='Demand point', 
                       color_discrete_sequence=px.colors.qualitative.G10,
                       facet_col_spacing=0.06, labels=names, category_orders=category_orders).data)

fig.update_layout(layout, yaxis=dict(tickformat='%'), legend_title_text='<b>No intervention :<br>Desalination: ')
update_labels(fig)
fig.show()
fig.write_image("unmetDemandDesalinationChtouka.png", scale=4, width=700, height=500)

In [ ]:
dff.groupby('Scenario')['production_kg'].sum()

In [ ]:
dff = df_crop.groupby(['Scenario', 'Year', 'crop'])[['production_kg']].sum() / 1000
dff = dff.reset_index()
fig = px.bar(dff, x='Year', y='production_kg', color='crop', facet_col='Scenario',
              color_discrete_sequence=px.colors.qualitative.T10,
              facet_col_spacing=0.06, labels=names, category_orders=category_orders)
fig.update_layout(layout)
update_labels(fig)
fig.show()
fig.write_image("productionDesalinationIWP.png", scale=4, width=900, height=400)

In [ ]:
dff_energy = df_main.groupby(['Scenario', 'Year', 'type']).agg({'swpa_e': 'sum'}) / 1000000
dff_energy.reset_index(inplace=True)
dff_energy.loc[dff_energy['type'].str.contains('SW|Aquifer'), 'type'] = 'Surface water'
dff_energy.loc[dff_energy['type'].str.contains('GW'), 'type'] = 'Groundwater pumping'
dff_energy.loc[dff_energy['type'].str.contains('Pipeline'), 'type'] = 'Surface water conveyance'
dff_energy.loc[dff_energy['type'].str.contains('DS'), 'type'] = 'Desalinated water conveyance'
dff_energy.loc[dff_energy['type'].str.contains('WWR'), 'type'] = 'Reused Wastewater'
dff_energy = dff_energy.loc[dff_energy['type'] != 'Transmission Pipeline']
df_wwt['type'] = 'Wastewater treatment'
df_desal['type'] = 'Desalination'
for df in [df_wwt, df_desal]:
    dff = df.groupby(['Scenario', 'Year', 'type']).agg({'swpa_e': 'sum'}) / 1000000
    dff.reset_index(inplace=True)
    dff_energy = dff_energy.append(dff, ignore_index=True)

dff = dff_energy.groupby(['Scenario', 'Year', 'type']).agg({'swpa_e': 'sum'}).reset_index()
dff = dff.loc[dff['swpa_e']!=0]
# dff = dff.loc[dff['Scenario']=='Reference']

fig = px.bar(dff, x='Year', y='swpa_e', color='type', facet_col='Scenario',
              color_discrete_sequence=px.colors.qualitative.Set2,
              facet_col_spacing=0.06, labels=names, category_orders=category_orders)
fig.update_layout(layout)
update_labels(fig)
fig.show()
fig.write_image("energyDemandDesalinationIWP.png", scale=4, width=900, height=500)
# fig.write_html("energyDemand.html", include_plotlyjs='cdn')

In [ ]:
dff_unmet = df_main.copy()
dff_unmet.loc[dff_unmet['type'].str.contains('Agriculture'), 'category'] = 'Agriculture'
dff_unmet.loc[dff_unmet['type'].str.contains('Domestic'), 'category'] = 'Domestic'
dff_unmet = dff_unmet.loc[~dff_unmet['type'].str.contains('Aquifer')]
dff_unmet = dff_unmet.loc[dff_unmet['type'] != 'Transmission Pipeline']
water_req_year = \
    dff_unmet.groupby(['Scenario', 'Year', 'Date', 'Demand point', 'category'])[
        'water_required'].mean().reset_index().groupby(
        ['Scenario', 'Year', 'category'])['water_required'].sum()
unment_demand = pd.DataFrame()
unment_demand['unmet_demand_year'] = 1 - (dff_unmet.groupby(['Scenario', 'Year', 'category'])['sswd'].sum() /
                              water_req_year)

dff = unment_demand.reset_index()
dff = dff

fig = px.line(dff.loc[dff['Scenario'] == 'Desalination'], 
              x='Year', y='unmet_demand_year', color='category', 
              color_discrete_sequence=px.colors.qualitative.G10,
              facet_col_spacing=0.06, labels=names, category_orders=category_orders)
fig.update_traces(line=dict(dash='dash'))

fig.add_traces(px.line(dff.loc[dff['Scenario'] == 'Increased Water Productivity'],
                       x='Year', y='unmet_demand_year', color='category', 
                       color_discrete_sequence=px.colors.qualitative.G10,
                       facet_col_spacing=0.06, labels=names, category_orders=category_orders).data)

fig.update_layout(layout, yaxis=dict(tickformat='%'), legend_title_text='<b>Desalination :<br>Desalination & Water productivity: ')
update_labels(fig)
fig.show()
fig.write_image("unmetDemandDesalinationIWP.png", scale=4, width=700, height=500)
# fig.write_html("unmetDemandAgriculture.html", include_plotlyjs='cdn')

In [ ]:
dff_wtd = df_main.copy()
# dff_wtd = dff_wtd.loc[dff_wtd['Climate'] == 'Climate change']
dff_wtd = dff_wtd.loc[dff_wtd['type'].str.contains('GW')]
wtd = dff_wtd.groupby(['Scenario', 'Date', 'Supply point'])['wtd'].mean().reset_index()

dff = wtd.reset_index()
dff = dff.loc[dff['Supply point'].isin([
#     'Chtouka GW', 
    'Souss GW'
])]

fig = px.line(dff.loc[dff['Scenario'] == 'Desalination'], 
              x='Date', y='wtd', color='Supply point', 
              color_discrete_sequence=px.colors.qualitative.G10,
              facet_col_spacing=0.06, labels=names, category_orders=category_orders)
fig.update_traces(line=dict(dash='dash'))

fig.add_traces(px.line(dff.loc[dff['Scenario'] == 'Increased Water Productivity'],
                       x='Date', y='wtd', color='Supply point', 
                       color_discrete_sequence=px.colors.qualitative.G10,
                       facet_col_spacing=0.06, labels=names, category_orders=category_orders).data)
layout_copy = layout.copy()
layout_copy['yaxis'] = dict(range=[dff.wtd.max()*1.05, dff.wtd.min()*0.95])
fig.update_layout(layout_copy, legend_title_text='<b>Desalination :<br>Desalination & Water productivity: ')
update_labels(fig)
fig.show()
fig.write_image("drawdownDesalinationIWPSouss.png", scale=4, width=700, height=500)

In [ ]:
dff_unmet = df_main.copy()
dff_unmet.loc[dff_unmet['type'].str.contains('Agriculture'), 'category'] = 'Agriculture'
dff_unmet.loc[dff_unmet['type'].str.contains('Domestic'), 'category'] = 'Domestic'
dff_unmet = dff_unmet.loc[~dff_unmet['type'].str.contains('Aquifer')]
dff_unmet = dff_unmet.loc[dff_unmet['type'] != 'Transmission Pipeline']
water_req_year = \
    dff_unmet.groupby(['Scenario', 'Climate', 'Year', 'Date', 'Demand point', 'category'])[
        'water_required'].mean().reset_index().groupby(
        ['Scenario', 'Climate', 'Year', 'category'])['water_required'].sum()
unment_demand = pd.DataFrame()
unment_demand['unmet_demand_year'] = 1 - (dff_unmet.groupby(['Scenario', 'Climate', 'Year', 'category'])['sswd'].sum() /
                              water_req_year)

dff = unment_demand.reset_index()
dff = dff

fig = px.line(dff.loc[dff['Climate'] == 'Historical climate'], 
              x='Year', y='unmet_demand_year', color='category', 
              color_discrete_sequence=px.colors.qualitative.G10,
              facet_col_spacing=0.06, labels=names, category_orders=category_orders)
fig.update_traces(line=dict(dash='dash'))

fig.add_traces(px.line(dff.loc[dff['Climate'] == 'Climate change'],
                       x='Year', y='unmet_demand_year', color='category', 
                       color_discrete_sequence=px.colors.qualitative.G10,
                       facet_col_spacing=0.06, labels=names, category_orders=category_orders).data)

fig.update_layout(layout, yaxis=dict(tickformat='%'), legend_title_text='<b>Historical climate :<br>Climate change: ')
update_labels(fig)
fig.show()
fig.write_image("unmetDemandReference.png", scale=4, width=700, height=500)
# fig.write_html("unmetDemandAgriculture.html", include_plotlyjs='cdn')

In [ ]:
dff_wtd = df_main.copy()
dff_wtd = dff_wtd.loc[dff_wtd['type'].str.contains('GW')]
wtd = dff_wtd.groupby(['Climate', 'Date', 'Supply point'])['wtd'].mean().reset_index()

dff = wtd.reset_index()
dff = dff.loc[dff['Supply point'].isin([
    'Chtouka GW', 
    'Souss GW'
])]

fig = px.line(dff.loc[dff['Climate'] == 'Historical climate'], 
              x='Date', y='wtd', color='Supply point', 
              color_discrete_sequence=px.colors.qualitative.G10,
              facet_col_spacing=0.06, labels=names, category_orders=category_orders)
fig.update_traces(line=dict(dash='dash'))

fig.add_traces(px.line(dff.loc[dff['Climate'] == 'Climate change'],
                       x='Date', y='wtd', color='Supply point', 
                       color_discrete_sequence=px.colors.qualitative.G10,
                       facet_col_spacing=0.06, labels=names, category_orders=category_orders).data)
layout_copy = layout.copy()
layout_copy['yaxis'] = dict(range=[dff.wtd.max()*1.05, dff.wtd.min()*0.95])
fig.update_layout(layout_copy, legend_title_text='<b>Historical climate :<br>Climate change: ')
update_labels(fig)
fig.show()
fig.write_image("drawdownReference.png", scale=4, width=700, height=500)


In [ ]:
dff_energy = df_main.groupby(['Scenario', 'Climate', 'Year']).agg({'swpa_e': 'sum'}) / 1000000
dff_energy.reset_index(inplace=True)
for df in [df_wwt, df_desal]:
    dff = df.groupby(['Scenario', 'Climate', 'Year']).agg({'swpa_e': 'sum'}) / 1000000
    dff.reset_index(inplace=True)
    dff_energy = dff_energy.append(dff, ignore_index=True)

dff = dff_energy.groupby(['Scenario', 'Climate', 'Year']).agg({'swpa_e': 'sum'}).reset_index()

fig = px.line(dff, x='Year', y='swpa_e', color='Climate', #facet_col='Climate',
              color_discrete_sequence=px.colors.qualitative.T10,
              facet_col_spacing=0.06, labels=names, category_orders=category_orders)
fig.update_layout(layout)
update_labels(fig)
fig.show()

In [ ]:
dff = df_main.loc[df_main['type'] != 'Transmission Pipeline']
dff = dff.groupby(['Scenario', 'Climate', 'Year']).agg({'sswd': lambda r: sum(r) / 1000000}).reset_index()

fig = px.line(dff, x='Year', y='sswd', color='Scenario', facet_col='Climate',
              color_discrete_sequence=px.colors.qualitative.T10,
              facet_col_spacing=0.06, labels=names,  category_orders=category_orders)
fig.update_layout(layout)
update_labels(fig)
fig.show()
# fig.write_image("waterDeliveries.png", scale=4, width=900, height=500)
# fig.write_html("waterDeliveries.html", include_plotlyjs='cdn')

In [ ]:
dff_unmet = df_main.copy()
dff_unmet.loc[dff_unmet['type'].str.contains('Agriculture'), 'category'] = 'Agriculture'
dff_unmet.loc[dff_unmet['type'].str.contains('Domestic'), 'category'] = 'Domestic'
dff_unmet = dff_unmet.loc[~dff_unmet['type'].str.contains('Aquifer')]
dff_unmet = dff_unmet.loc[dff_unmet['type'] != 'Transmission Pipeline']
water_req_year = \
    dff_unmet.groupby(['Scenario', 'Climate', 'Year', 'Date', 'Demand point', 'category'])[
        'water_required'].mean().reset_index().groupby(
        ['Scenario', 'Climate', 'Year', 'category'])['water_required'].sum()
unment_demand = pd.DataFrame()
unment_demand['unmet_demand_year'] = 1 - (dff_unmet.groupby(['Scenario', 'Climate', 'Year', 'category'])['sswd'].sum() /
                              water_req_year)

dff = unment_demand.reset_index()
dff = dff.loc[dff['category'] == 'Agriculture']

fig = px.line(dff, x='Year', y='unmet_demand_year', color='Scenario', facet_col='Climate',
              color_discrete_sequence=px.colors.qualitative.T10,
              facet_col_spacing=0.06, labels=names, category_orders=category_orders)
fig.update_layout(layout, yaxis=dict(tickformat='%'))
update_labels(fig)
fig.show()
fig.write_image("unmetDemandAgriculture.png", scale=4, width=900, height=500)
fig.write_html("unmetDemandAgriculture.html", include_plotlyjs='cdn')

In [ ]:
dff = unment_demand.reset_index()
dff = dff.loc[dff['category'] == 'Domestic']

fig = px.line(dff, x='Year', y='unmet_demand_year', color='Scenario', facet_col='Climate',
              color_discrete_sequence=px.colors.qualitative.T10,
              facet_col_spacing=0.06, labels=names, category_orders=category_orders)
fig.update_layout(layout, yaxis=dict(tickformat='%'))
update_labels(fig)
fig.show()
fig.write_image("unmetDemandDomestic.png", scale=4, width=900, height=500)
fig.write_html("unmetDemandDomestic.html", include_plotlyjs='cdn')

In [ ]:
dff = df_crop.groupby(['Scenario', 'Climate', 'Year']).agg({'production_kg': lambda r: sum(r) / 1000000}).reset_index()

fig = px.line(dff, x='Year', y='production_kg', color='Scenario', facet_col='Climate',
              color_discrete_sequence=px.colors.qualitative.T10, 
              facet_col_spacing=0.06, labels=names, category_orders=category_orders)
fig.update_layout(layout)
update_labels(fig)
fig.show()
fig.write_image("cropProduction.png", scale=4, width=900, height=500)
fig.write_html("cropProduction.html", include_plotlyjs='cdn')

In [ ]:
dff_energy = df_main.groupby(['Scenario', 'Climate', 'Year']).agg({'swpa_e': 'sum'}) / 1000000
dff_energy.reset_index(inplace=True)
for df in [df_wwt, df_desal]:
    dff = df.groupby(['Scenario', 'Climate', 'Year']).agg({'swpa_e': 'sum'}) / 1000000
    dff.reset_index(inplace=True)
    dff_energy = dff_energy.append(dff, ignore_index=True)

dff = dff_energy.groupby(['Scenario', 'Climate', 'Year']).agg({'swpa_e': 'sum'}).reset_index()

fig = px.line(dff, x='Year', y='swpa_e', color='Scenario', facet_col='Climate',
              color_discrete_sequence=px.colors.qualitative.T10,
              facet_col_spacing=0.06, labels=names, category_orders=category_orders)
fig.update_layout(layout)
update_labels(fig)
fig.show()
fig.write_image("energyDemand.png", scale=4, width=900, height=500)
fig.write_html("energyDemand.html", include_plotlyjs='cdn')

In [ ]:
folder = '../Morocco dashboard'
scenario1 = 'Reference'
scenario2 = 'Desalination Wastewater Reuse'
# scenario3 = 'Reference Wastewater Reuse'
# scenario4 = 'Desalination Wastewater Reuse'
climate = 'Climate Change'
# climate_title = f'Historical climate {climate}'
climate_title = climate
folder1 = f"{folder}/data/{scenario1}/{climate}/"
folder2 = f"{folder}/data/{scenario2}/{climate}/"
scenario2 = 'Desalination & Wastewater Reuse'
# folder3 = f"{folder}/data/{scenario3}/{climate}/"
# folder4 = f"{folder}/data/{scenario4}/{climate}/"
df1 = pd.read_csv(f"{folder1}/results.gz")
df2 = pd.read_csv(f"{folder2}/results.gz")
# df3 = pd.read_csv(f"{folder3}/results.gz")
# df4 = pd.read_csv(f"{folder4}/results.gz")
df1['Scenario'] = scenario1
df2['Scenario'] = scenario2
# df3['Scenario'] = scenario1
# df4['Scenario'] = scenario2
df1['Reuse'] = 'No'
df2['Reuse'] = 'Yes'
# df3['Reuse'] = 'Yes'
# df4['Reuse'] = 'Yes'
df_energy = df1.append([df2], ignore_index=True)
df_energy = df_energy.loc[df_energy.Year >= 2020]

#desal data
df1 = pd.read_csv(f"{folder1}/desal_data.gz")
df2 = pd.read_csv(f"{folder2}/desal_data.gz")
# df3 = pd.read_csv(f"{folder3}/desal_data.gz")
# df4 = pd.read_csv(f"{folder4}/desal_data.gz")
df1['Scenario'] = scenario1
df2['Scenario'] = scenario2
# df3['Scenario'] = scenario1
# df4['Scenario'] = scenario2
df1['Reuse'] = 'No'
df2['Reuse'] = 'Yes'
# df3['Reuse'] = 'Yes'
# df4['Reuse'] = 'Yes'
df_desal = df1.append([df2], ignore_index=True)
df_desal = df_desal.loc[df_desal.Year >= 2020]

#wwt data
df1 = pd.read_csv(f"{folder1}/wwtp_data.gz")
df2 = pd.read_csv(f"{folder2}/wwtp_data.gz")
# df3 = pd.read_csv(f"{folder3}/wwtp_data.gz")
# df4 = pd.read_csv(f"{folder4}/wwtp_data.gz")
df1['Scenario'] = scenario1
df2['Scenario'] = scenario2
# df3['Scenario'] = scenario1
# df4['Scenario'] = scenario2
df1['Reuse'] = 'No'
df2['Reuse'] = 'Yes'
# df3['Reuse'] = 'Yes'
# df4['Reuse'] = 'Yes'
df_wwtp = df1.append([df2], ignore_index=True)
df_wwtp = df_wwtp.loc[df_wwtp.Year >= 2020]

In [ ]:
layout = dict(
    autosize=True,
    height=350,
    margin=dict(l=0, r=20, b=50, t=100),
    hovermode="closest",
    plot_bgcolor="#fff",
    paper_bgcolor="#fff",
    legend=dict(font=dict(size=12), orientation="h", title=''),
    xaxis={'title': ''},
    showlegend=True,
    hoverlabel={'align': 'left'},
    font=dict(size=14, color="#7f7f7f"),
    yaxis={'title': ''},
)

In [ ]:
df = energy_demand_plot(df_energy, df_wwtp, df_desal, 'Year')

fig = px.bar(df, x='Year', y='swpa_e', color='type', facet_col='Scenario', #facet_row='Reuse',
              labels={"swpa_e": "Energy (GWh)"}, #title='',
              facet_col_spacing=0.06, color_discrete_sequence=px.colors.qualitative.T10)
# fig.update_layout(legend=dict(
#     orientation="h",
#     yanchor="top",
#     y=-0.15,
#     xanchor="left",
#     x=0
# ))
for axis in fig.layout:
    if type(fig.layout[axis]) == go.layout.YAxis:
        fig.layout[axis].title.text = ''
    if type(fig.layout[axis]) == go.layout.XAxis:
        fig.layout[axis].title.text = ''
fig.update_layout(layout)
fig.show()
# fig.write_image('EnergyDemandSoussMassa.png', scale=4)

In [ ]:
df = water_supply_plot(df_energy, 'Year')

fig = px.bar(df, x='Year', y='sswd', color='type', facet_col='Scenario', #facet_row='Reuse',
              labels={"sswd": "Water (Mm<sup>3</sup>)"}, #title='Water supply in Souss-Massa',
              facet_col_spacing=0.06, color_discrete_sequence=px.colors.qualitative.Set2)
for axis in fig.layout:
    if type(fig.layout[axis]) == go.layout.YAxis:
        fig.layout[axis].title.text = ''
    if type(fig.layout[axis]) == go.layout.XAxis:
        fig.layout[axis].title.text = ''
fig.update_layout(layout, title='Water supply (Mm<sup>3</sup>)')
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.for_each_annotation(lambda a: a.update(font_size=16))
fig.show()
file_name='my_file.pkl'
f = open(file_name,'wb')
pickle.dump(fig,f)
f.close()
fig.write_image('WaterDemandSoussMassa.png', scale=4)

In [ ]:
df_energy

In [ ]:
df = unmet_demand_plot(df_energy, 'Year')
df = df.loc[df['category']=='Domestic']
reuse = {'Yes': ' with reuse', 'No': '',}
# df['FullScenario'] = df['Scenario'] + [reuse[i] for i in df['Reuse']]

fig = px.line(df, x='Year', y='value', color='Scenario', #facet_col='Reuse', #facet_row='Reuse',
              labels={"value": "Unmet demand (%)", 'FullScenario': 'Scenario'}, #title='Unmet demand in agriculture Souss-Massa',
              facet_col_spacing=0.06, color_discrete_sequence=px.colors.qualitative.T10)
for axis in fig.layout:
    if type(fig.layout[axis]) == go.layout.YAxis:
        fig.layout[axis].title.text = ''
    if type(fig.layout[axis]) == go.layout.XAxis:
        fig.layout[axis].title.text = ''
fig.update_layout(layout)
fig.update_layout(yaxis={'tickformat': "%"})
fig.show()
# fig.write_image('UnmetDemandSoussMassa.pdf', width=900)

In [ ]:
def wtd_plot(water_delivered, time_frame, layout):
    dff_wtd = water_delivered.copy()
    dff_wtd = dff_wtd.loc[dff_wtd['type'].str.contains('GW')]
    wtd = dff_wtd.groupby(['Scenario', time_frame, 'Supply point'])['wtd'].mean().reset_index()
    df = wtd

    fig = px.line(df, x=time_frame, y='wtd', color='Supply point', facet_col='Scenario',
                  color_discrete_sequence=px.colors.qualitative.Vivid)
    fig.update_traces(hovertemplate='<b>Value</b>: %{y:.2f}' + '<br><b>Year</b>: %{x}')
    fig.update_layout(layout,
                      yaxis=dict(range=[df.wtd.max()*1.05, df.wtd.min()*0.95]))
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
    for axis in fig.layout:
        if type(fig.layout[axis]) == go.layout.YAxis:
            fig.layout[axis].title.text = ''
        if type(fig.layout[axis]) == go.layout.XAxis:
            fig.layout[axis].title.text = ''

    return fig

In [ ]:
fig = wtd_plot(df_energy.loc[df_energy['Supply point']=='Chtouka GW'], 'Date', layout)
fig.write_image('ChtoukaDepth.png', scale=4)

In [ ]:
def crop_production(df, group_by, layout):
    dff = df.groupby(['Scenario', 'Year', group_by])[['production_kg']].sum() / 1000
    dff = dff.reset_index()
    fig = px.bar(dff, x='Year', y='production_kg', color=group_by, facet_col='Scenario',
                  color_discrete_sequence=px.colors.qualitative.T10,
                  facet_col_spacing=0.06,
                  labels={'production_kg': 'Production (ton)'})

    fig.update_layout(layout)
    for axis in fig.layout:
        if type(fig.layout[axis]) == go.layout.YAxis:
            fig.layout[axis].title.text = ''
        if type(fig.layout[axis]) == go.layout.XAxis:
            fig.layout[axis].title.text = ''
    return fig

In [ ]:
df1 = pd.read_csv(os.path.join(folder1, 'production.gz'))
df1['Scenario'] = scenario1
df2 = pd.read_csv(os.path.join(folder2, 'production.gz'))
df2['Scenario'] = scenario2

dff = df1.append(df2, ignore_index=True)
fig = crop_production(dff, 'crop', layout)
fig.write_image('CropProduction.png', scale=4)

In [ ]:
def water_delivered_plot(water_delivered, time_frame, layout):
    dff_delivered = water_delivered.copy()
    dff_delivered.loc[dff_delivered['type'].str.contains('Agriculture'), 'category'] = 'Agriculture'
    dff_delivered.loc[dff_delivered['type'].str.contains('Domestic'), 'category'] = 'Domestic'
    dff_delivered.loc[dff_delivered['type'].str.contains('Aquifer'), 'category'] = 'Aquifer recharge'
    dff_delivered.loc[dff_delivered['type'].str.contains('SW|Aquifer'), 'type'] = 'Surface water'
    dff_delivered.loc[dff_delivered['type'].str.contains('GW'), 'type'] = 'Groundwater'
    dff_delivered.loc[dff_delivered['type'].str.contains('DS'), 'type'] = 'Desalinated water'
    dff_delivered.loc[dff_delivered['type'].str.contains('WWR'), 'type'] = 'Wastewater reuse'
    dff_delivered = dff_delivered.loc[dff_delivered['type'] != 'Transmission Pipeline']
    dff_delivered = dff_delivered.groupby(['Scenario', time_frame, 'category', 'type'])['sswd'].sum() / 1000000
    dff_delivered = dff_delivered.reset_index()

    dff_delivered_cat = dff_delivered.groupby(['Scenario', time_frame, 'category']).sswd.sum()
    dff_delivered['share'] = dff_delivered.sswd / dff_delivered.set_index(['Scenario', time_frame, 'category']).index.map(
        dff_delivered_cat)

    df = dff_delivered_cat.reset_index()
    fig = px.bar(df, x=time_frame, y='sswd', color='category', facet_col='Scenario',
                  color_discrete_sequence=px.colors.qualitative.Dark2)
    # fig.update_traces(fill='tonexty')
    fig.update_layout(layout)
    for axis in fig.layout:
        if type(fig.layout[axis]) == go.layout.YAxis:
            fig.layout[axis].title.text = ''
        if type(fig.layout[axis]) == go.layout.XAxis:
            fig.layout[axis].title.text = ''
    return fig

In [ ]:
fig = water_delivered_plot(df_energy, 'Year', layout)
fig.write_image('WaterDelivered.png', scale=4)

In [ ]:
### adjust the variables from this bell accordingly to read the data
w_rate = 0
pv_rate = 0
grid_rate = -0
scenario = 'Desalination'
climate = 'Climate Change'
folder = f"test dash results/data/{scenario}/{climate}/W{w_rate}_PV{pv_rate}_Grid{grid_rate}"
df = pd.read_csv(f"{folder}/lcoe.gz")

In [ ]:
scenario = 'bau'
pv = 'high'
folder = f"test dash results/Butane_calculations/{scenario}/{pv}_PV"
df = pd.read_csv(f"{folder}/results.csv")

In [ ]:
import json
import geopandas as gpd

provinces = gpd.read_file('Data/GIS/Admin/Provinces.gpkg', encoding='utf-8')

with open(r'Data\GIS\Processed layers\cropland.geojson') as response:
    cropland = json.load(response)
    for feature in cropland['features']:
        # print(feature)
        feature['id'] = feature['properties']['id']

In [ ]:
token = "pk.eyJ1IjoiY2FtaWxvcmciLCJhIjoiY2p1bTl0MGpkMjgyYjQ0b2E0anRibWJ1MSJ9.GhUUGD6gok1d36lvP17CQQ"

In [ ]:
df['pv_energy_per_m2'] = df['pv_elec(KWh)'] / df['area_m2']

In [ ]:
import plotly.graph_objects as go
import itertools

var = 'cap_a(MW)'
year = 2020
legend_titles = {'pv_demand(KWh)': "PV generation (KWh)", 
                 'pv_energy_per_m2': 'PV generation per m<sup>2</sup> (KWh/m<sup>2</sup>)',
                 'srad': 'Solar radiation (kJ/(m<sup>2</sup>.day))',
                 'GWdepth': 'Groundwater depth (mbgl)',
                 'cap_a(MW)': 'PV capacity (MW)'}
var_filters = {'pv_demand(KWh)': (df.Year==year), 
               'pv_energy_per_m2': (df.Year==year) & (df[var]<=2),
               'srad': (df.Year==year),
               'GWdepth': (df.Year==year) & (df[var]<=500),
               'cap_a(MW)': (df.Year==year)}

legend_title = legend_titles[var]
var_filter = var_filters[var]
dff = df.loc[var_filter, ['Year','Demand point', var]].dropna()

colorscales = [
    ((0.0, '#636efa'), (1.0, '#636efa')),
    ((0.0, '#EF553B'), (1.0, '#EF553B')),
    ((0.0, '#00cc96'), (1.0, '#00cc96'))
]

fig = go.Figure()

fig.add_scattermapbox(mode='lines',
                line=dict(color='rgb(80,100,80)', width=1),
                fill='toself',
                fillcolor='rgba(80,100,80,0.1)',
                below=True,
                lon=list(itertools.chain.from_iterable(list(itertools.chain.from_iterable(
                        [[list(l.coords.xy[0]) + [None] for l in line.boundary] for line in provinces.geometry])))),
                lat=list(itertools.chain.from_iterable(list(itertools.chain.from_iterable(
                        [[list(l.coords.xy[1]) + [None] for l in line.boundary] for line in provinces.geometry])))),
                hoverinfo='skip',
                showlegend=False)

# for i, tech in enumerate(dff['least_cost_technology'].unique()):
#     dfp = dff[dff['least_cost_technology'] == tech]
#     fig.add_choroplethmapbox(geojson=cropland, locations=dfp['Demand point'],
#                        z=[i,] * len(dfp),
#                        showlegend=True,
#                        colorscale=colorscales[i], showscale=False,
#                        marker_line_width=0.1, name=tech,
#                        hoverinfo='skip')


fig.add_choroplethmapbox(
    locations=dff['Demand point'], # Spatial coordinates
    z = dff[var], # Data to be color-coded
    geojson = cropland, # set of locations match entries in `locations`
    marker_line_width=0.1,
#     hoverinfo='skip',
    colorbar_title = legend_title,
)

fig.update_layout(mapbox_style="light", mapbox_accesstoken=token, mapbox_center={"lon": -8.9, 'lat': 30.2}, mapbox_zoom=7)

# fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()
fig.show(renderer="browser")

In [ ]:
import plotly.graph_objects as go
import itertools

# dff1 = df.loc[df.Year==2020].dropna()
# dff2 = df.loc[df.Year==2030].dropna()

colorscales = [
    ((0.0, '#636efa'), (1.0, '#636efa')),
    ((0.0, '#EF553B'), (1.0, '#EF553B')),
    ((0.0, '#00cc96'), (1.0, '#00cc96'))
]

fig_dict = {
    "data": [],
    "layout": {},
    "frames": []
}

# fig_dict["layout"]["updatemenus"] = [
#     {
#         "buttons": [
#             {
#                 "args": [None, {"frame": {"duration": 1500, "redraw": True},
#                                 "fromcurrent": True, "transition": {"duration": 1500,
#                                                                     "easing": "quadratic-in-out"}}],
#                 "label": "Play",
#                 "method": "animate"
#             },
#             {
#                 "args": [[None], {"frame": {"duration": 0, "redraw": True},
#                                   "mode": "immediate",
#                                   "transition": {"duration": 0}}],
#                 "label": "Pause",
#                 "method": "animate"
#             }
#         ],
#         "direction": "left",
#         "pad": {"r": 10, "t": 40},
#         "showactive": False,
#         "type": "buttons",
#         "x": 0.1,
#         "xanchor": "right",
#         "y": 0,
#         "yanchor": "top"
#     }
# ]

sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Year:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 800, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 20},
    "len": 0.8,
    "x": 0.1,
    "y": 0,
    "steps": []
}

dff = df.loc[df.Year==2020, ['Year','Demand point','least_cost_technology']].dropna()
for i, tech in enumerate(dff['least_cost_technology'].unique()):
    dfp = dff[dff['least_cost_technology'] == tech]
    data_dict = dict(geojson=cropland, locations=dfp['Demand point'],
                 z=[i,] * len(dfp), showlegend=True,
                 colorscale=colorscales[i], showscale=False,
                 marker_line_width=0.1, name=tech,
                 hoverinfo='skip', type="choroplethmapbox")
    fig_dict["data"].append(data_dict)

fig_dict["data"].append(dict(mode='lines', type='scattermapbox',
                            line=dict(color='rgb(80,100,80)', width=1),
                            fill='toself',
                            fillcolor='rgba(80,100,80,0.1)',
                            below=True,
                            lon=list(itertools.chain.from_iterable(list(itertools.chain.from_iterable(
                                    [[list(l.coords.xy[0]) + [None] for l in line.boundary] for line in provinces.geometry])))),
                            lat=list(itertools.chain.from_iterable(list(itertools.chain.from_iterable(
                                    [[list(l.coords.xy[1]) + [None] for l in line.boundary] for line in provinces.geometry])))),
                            hoverinfo='skip',
                            showlegend=False))

# years = range(2020,2051, 5)
years = [2029, 2030, 2031, 2032]

# data = []
for year in years:
    frame = {"data": [], "name": str(year)}
    dff = df.loc[df.Year==year, ['Year','Demand point','least_cost_technology']].dropna()
    for i, tech in enumerate(dff['least_cost_technology'].unique()):
        dfp = dff[dff['least_cost_technology'] == tech]
        data_dict = dict(geojson=cropland, locations=dfp['Demand point'],
                     z=[i,] * len(dfp), showlegend=True,
                     colorscale=colorscales[i], showscale=False,
                     marker_line_width=0.1, name=tech,
                     hoverinfo='skip', type="choroplethmapbox")
        frame["data"].append(data_dict)
    
    fig_dict["frames"].append(frame)
    slider_step = {"args": [
        [year],
        {"frame": {"duration": 800, "redraw": True},
         "mode": "immediate",
         "transition": {"duration": 800}}
    ],
        "label": f'{year}',
        "method": "animate"}
    sliders_dict["steps"].append(slider_step)
    
fig_dict["layout"]["sliders"] = [sliders_dict]

fig = go.Figure(fig_dict)

# fig = go.Figure(data=[go.Choroplethmapbox(
#     locations=dff1['Demand point'], # Spatial coordinates
#     z = dff1['lcoe'], # Data to be color-coded
#     geojson = cropland, # set of locations match entries in `locations`
#     marker_line_width=0.1,
#     hoverinfo='skip',
#     colorbar_title = "LCOE ($/kWh)",
# ),
#     go.Choroplethmapbox(
#     locations=dff2['Demand point'], # Spatial coordinates
#     z = dff2['lcoe'], # Data to be color-coded
#     geojson = cropland, # set of locations match entries in `locations`
#     marker_line_width=0.1,
#     hoverinfo='skip',
#     colorbar_title = "LCOE ($/kWh)",
# )])

# steps = []

# for i, year in enumerate(years):
#     step = dict(method='restyle',
#                 args=['visible', [False] * len(years)],
#                 label=str(year)) # label to be displayed for each step (year)
#     step['args'][1][i] = True
#     steps.append(step)

# sliders = [dict(active=0, pad={"t": 1}, steps=steps, currentvalue={"prefix": "Year: "})]  
# fig = go.Figure(data=data)

fig.update_layout(mapbox_style="light", mapbox_accesstoken=token, mapbox_center={"lon": -8.9, 'lat': 30.2}, mapbox_zoom=7)

# fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":2,"t":8,"l":5,"b":0})
# fig.show()
fig.show(renderer="browser")